# PANDAS

In [4]:
import pandas as pd
import numpy as np

In [61]:
dateparse = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')
#http://strftime.org/

In [68]:
trans = pd.read_csv("data/trans.csv", parse_dates = ["Date"], date_parser = dateparse)

In [63]:
print trans.head(3)
trans.dtypes

   TransID  Store  CustID       Date  Amount
0   100001   7254     418 2017-05-14    7337
1   100002   7055     537 2017-04-03    5705
2   100003   7012     395 2018-02-16    8506


TransID             int64
Store               int64
CustID              int64
Date       datetime64[ns]
Amount              int64
dtype: object

In [ ]:
#changing dtypes
trans["Date"] = trans["Date"].astype(str)
trans["CustID"] = trans["CustID"].astype(str)

### Subsetting and Indexing

In [ ]:
#substting entire columns

print "The Date column is -"
print trans["Date"]

In [ ]:
#subsetting a single value from a column
print "Value at the 8th index (9th position) from Date is"
print trans["Date"][8]

print"\n"
#subsetting a part of a column
print "Date values with index 10-15"
print trans["Date"][10:15]


In [ ]:
#using iloc

subset = trans.iloc[10:15, 1:4]
subset

In [ ]:
#using loc

#loc uses row/column names (labels) (a.k.a index) to select data
#this is unlike iloc that used positions to select data

#trans_label is just trans with TransID as row labels (index)
trans_label = trans.set_index("TransID")
print trans_label.head(5)
print("\n")

#subset
print trans_label.loc[100003:100013, "CustID":"Amount"]

In [ ]:
#to filter out TransID and Date where Trans amount is > 9900

trans[trans["Amount"]>9900].loc[:,["TransID","Date"]]

In [ ]:
#to get all transactions on a particular date - June 2, 2017
trans[trans["Date"]==pd.datetime.strptime("02/06/2017", "%d/%m/%Y")]

#trans[trans["Date"]=="2017-06-02]

In [ ]:
#to get all those transactions whose amount is greater than 2.2 times the average transaction amount

trans[trans["Amount"]>=2.2*trans["Amount"].mean()]

In [ ]:
trans.axes

### Group By (Aggregation)

In [ ]:
#this statement creates a 'groupby' object - an abstract object that can be used to perform aggregations
#the 'by' parameter can be a list of columns as well

g = trans.groupby(by = "Store", as_index=False)

#help(pd.DataFrame.groupby)

In [ ]:
#further we can apply functions to columns in the groupby object

store_amt = g["Amount"].sum()
store_lasttrans = g["Date"].max()

#store_amt
#store_lasttrans

In [ ]:
#if the function has to be applied to ALL columns in the groupby object

store_sum = g.sum()

In [ ]:
#if different functions have to be applied to different columns

g = trans.groupby(["Store", "CustID"], as_index=False).agg({"Amount":(sum),
                                "Date":(max)})

In [ ]:
#the groupby object can also be used to emulate what fixed calculations in tableau (partition by in SQL) does
#egs - if the percent contribution of each transaction to the Sales of the Store is required

trans["Amt_Contribution"] = trans.groupby("Store", as_index=False)["Amount"].transform(lambda x: x*100/sum(x))

#check
#trans[trans["Store"]==7254]

### Merging (Joins)

In [ ]:
#concat is equivalent to rbind/cbind in R (i.e. it appends rows/columns)

#we will append this single row to the trans table
df = pd.DataFrame([[9999,7000,9999,pd.datetime.strptime("02/06/1993", "%d/%m/%Y"), 10.5, 100]],
                 columns = trans.columns)
#df

#concat step
df_new = pd.concat([trans, df], axis=0, ignore_index=True)
#df_new

In [ ]:
#'merge' is a single method designed to take care of all joining needs

#pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
#         left_index=False, right_index=False, sort=True,
#         suffixes=('_x', '_y'), copy=True, indicator=False,
#         validate=None)

#left, right - dataframes
#how - 'inner' or 'outer' or 'left' or 'right'
#on - columns (names) to join on (as long as column names are available in both df's)
#left_on, right_on - if join keys have different names in both frames
#left_index, right_index - boolean parameter indicating if indices should be used as join keys
#sort - boolean parameter indicating if sorting of results in required (MAJOR PERFORMANCE CAVEAT! Set to False preferably)
#suffixes - suffixes for columns (not keys) that are present in both df's
#indicator - if True, another column is added to output indicating whether the row comes from left or right or both df(s)
#validate - avaliable in pandas 0.21.1 and up - checks uniqueness of columns in case of one-one/one-many/many-one merges

In [ ]:
#read another table
store_map = pd.read_csv("data/geog.csv", encoding='latin1', engine="python")

new = pd.merge(trans, store_map, on="Store", how = "left", sort=False)

new

### Reshaping (Long-Wide and Wide-Long) (Casting and Melting)

In [5]:
#long to wide (casting)

#read data
long_simple = pd.read_csv("data/long_simple.csv", encoding="latin1", engine="python")
long_simple

,Student ID,Subject,Marks
0,Justa Passman,Maths,23
1,Gina Cookson,Maths,11
2,Isidro Haber,Maths,4
3,Yetta Vivas,Maths,17
4,Dane Noland,Maths,15
5,Aide Fairfield,Maths,40
6,Tatiana Guardiola,Maths,30
7,Tana Fazzino,Maths,23
8,Elly Zemke,Maths,1
9,Hans Arteaga,Maths,1


In [6]:
wide_simple = long_simple.pivot_table(index="Student ID", columns="Subject", values = "Marks", aggfunc = sum)
#now the subjects are in columns
wide_simple

Subject,English,Hindi,Maths,Science
Student ID,,,,
Aide Fairfield,5,34,40,49
Dane Noland,25,12,15,11
Elly Zemke,4,21,1,35
Gina Cookson,27,49,11,14
Hans Arteaga,45,43,1,11
Herb Salido,5,12,40,32
Isidro Haber,34,45,4,15
Justa Passman,6,43,23,32
Kasey Horrigan,46,16,43,26


In [ ]:
#however, 'Student ID' is now an index
print wide_simple.iloc[0,0]

In [7]:
#we can remove 'Student ID' from the index
#a dummy index - range(0, N) will be created by default
wide_simple = wide_simple.reset_index()
wide_simple

Subject,Student ID,English,Hindi,Maths,Science
0,Aide Fairfield,5,34,40,49
1,Dane Noland,25,12,15,11
2,Elly Zemke,4,21,1,35
3,Gina Cookson,27,49,11,14
4,Hans Arteaga,45,43,1,11
5,Herb Salido,5,12,40,32
6,Isidro Haber,34,45,4,15
7,Justa Passman,6,43,23,32
8,Kasey Horrigan,46,16,43,26
9,Rosemarie Jorstad,35,36,25,9


In [10]:
#wide to long
long_simple_back = pd.melt(wide_simple, id_vars="Student ID", value_name="Marks", var_name="Subject")
long_simple_back

,Student ID,Subject,Marks
0,Aide Fairfield,English,5
1,Dane Noland,English,25
2,Elly Zemke,English,4
3,Gina Cookson,English,27
4,Hans Arteaga,English,45
5,Herb Salido,English,5
6,Isidro Haber,English,34
7,Justa Passman,English,6
8,Kasey Horrigan,English,46
9,Rosemarie Jorstad,English,35


#### What if there are more than two levels? (p.s. currently the data has 2 levels - Student and Subject)
#### Egs - Say the same 4 subjects are offered in multiple years (Year1 and Year2)


In [37]:
#long to wide

#read long data
long_data = pd.read_csv("data/long_data.csv", encoding="latin1", engine="python") 

In [38]:
#set Student ID, Year and Subject as indices
long_data = long_data.set_index(["Student ID", "Year", "Subject"])
long_data

[MultiIndex(levels=[[u'Aide Fairfield', u'Dane Noland', u'Elly Zemke', u'Gina Cookson', u'Hans Arteaga', u'Herb Salido', u'Isidro Haber', u'Justa Passman', u'Kasey Horrigan', u'Rosemarie Jorstad', u'Sabina Rhoda', u'Tana Fazzino', u'Tatiana Guardiola', u'Wendy Shelman', u'Yetta Vivas'], [u'Junior', u'Sophomore'], [u'English', u'Hindi', u'Math', u'Science']],
            labels=[[7, 3, 6, 14, 1, 0, 12, 11, 2, 4, 10, 13, 8, 9, 5, 7, 3, 6, 14, 1, 0, 12, 11, 2, 4, 10, 13, 8, 9, 5, 7, 3, 6, 14, 1, 0, 12, 11, 2, 4, 10, 13, 8, 9, 5, 7, 3, 6, 14, 1, 0, 12, 11, 2, 4, 10, 13, 8, 9, 5, 7, 3, 6, 14, 1, 0, 12, 11, 2, 4, 10, 13, 8, 9, 5, 7, 3, 6, 14, 1, 0, 12, 11, 2, 4, 10, 13, 8, 9, 5, 7, 3, 6, 14, 1, 0, 12, 11, 2, 4, 10, 13, 8, 9, 5, 7, 3, 6, 14, 1, 0, 12, 11, 2, 4, 10, 13, 8, 9, 5], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [50]:
#unstack your data on the required level ([1,2] i.e 'Year' and 'Subject')
wide_data = long_data.unstack([1,2])
wide_data

Marks                                                      
Year              Junior                       Sophomore                      
Subject             Math Science Hindi English      Math Science Hindi English
Student ID                                                                    
Aide Fairfield        35      23    30      40        18      42    31       7
Dane Noland           32      37     8      50         8      49    49      26
Elly Zemke            40      10    47      41        30      26    21      27
Gina Cookson          25       0    35       9        47      38    15       7
Hans Arteaga          17      30     2       5        17       9    27      13
Herb Salido           43      15    23      46        15      50    50      19
Isidro Haber          28      35    13      45        21      10     1      33
Justa Passman         36      21    47      37         0      24    48      20
Kasey Horrigan        34      16    30       5         4      48     5      26
Rosemarie Jorstad     41      26     1      32        45      17    42      18
Sabina Rhoda          37      33    16      41        20      31    28      43
Tana Fazzino          22       7    41      39        29      33    30      17
Tatiana Guardiola     50      20    47      11        28      22    25       1
Wendy Shelman         29      21     8       0        50      23    35      15
Yetta Vivas           24      27    44      15        16      21    48      50

In [52]:
#wide to long
long_data_back = wide_data.stack([1,2])
long_data_back

Marks
Student ID        Year      Subject       
Aide Fairfield    Junior    English     40
                            Hindi       30
                            Math        35
                            Science     23
                  Sophomore English      7
                            Hindi       31
                            Math        18
                            Science     42
Dane Noland       Junior    English     50
                            Hindi        8
                            Math        32
                            Science     37
                  Sophomore English     26
                            Hindi       49
                            Math         8
                            Science     49
Elly Zemke        Junior    English     41
                            Hindi       47
                            Math        40
                            Science     10
                  Sophomore English     27
                            Hindi       21
                            Math        30
                            Science     26
Gina Cookson      Junior    English      9
                            Hindi       35
                            Math        25
                            Science      0
                  Sophomore English      7
                            Hindi       15
...                                    ...
Tana Fazzino      Junior    Math        22
                            Science      7
                  Sophomore English     17
                            Hindi       30
                            Math        29
                            Science     33
Tatiana Guardiola Junior    English     11
                            Hindi       47
                            Math        50
                            Science     20
                  Sophomore English      1
                            Hindi       25
                            Math        28
                            Science     22
Wendy Shelman     Junior    English      0
                            Hindi        8
                            Math        29
                            Science     21
                  Sophomore English     15
                            Hindi       35
                            Math        50
                            Science     23
Yetta Vivas       Junior    English     15
                            Hindi       44
                            Math        24
                            Science     27
                  Sophomore English     50
                            Hindi       48
                            Math        16
                            Science     21

[120 rows x 1 columns]

### Rolling Aggregations

In [66]:
#sort values first
trans = trans.sort_values(["Store", "Date"])

#use of 'rolling' method
trans["Cumulative_Amt"] = trans["Amount"].rolling(window=2).sum()

In [132]:
#rolling sum repeating by partition

#read data and sort
data = pd.read_csv("data/rolling.csv", encoding="latin1", engine="python")
data = data.sort_values(["colA", "colB", "colC"])

#apply rolling to the groupby object to get rolling sum restarting in every partition
rolling_value = data.groupby(["colA", "colB"])["value"].rolling(window=2).sum()

#concat this with data
data["values_rolled_2"] = rolling_value.values

### Apply

In [134]:
#this is very similar to apply in R
help(pd.DataFrame.apply)

Help on method apply in module pandas.core.frame:

apply(self, func, axis=0, broadcast=False, raw=False, reduce=None, args=(), **kwds) unbound pandas.core.frame.DataFrame method
    Applies function along input axis of DataFrame.
    
    Objects passed to functions are Series objects having index
    either the DataFrame's index (axis=0) or the columns (axis=1).
    Return type depends on whether passed function aggregates, or the
    reduce argument if the DataFrame is empty.
    
    Parameters
    ----------
    func : function
        Function to apply to each column/row
    axis : {0 or 'index', 1 or 'columns'}, default 0
        * 0 or 'index': apply function to each column
        * 1 or 'columns': apply function to each row
    broadcast : boolean, default False
        For aggregation functions, return object of same size with values
        propagated
    raw : boolean, default False
        If False, convert each row or column into a Series. If raw=True the
        passed f

In [140]:
#read data
data = pd.read_csv("data/apply.csv", encoding="latin1", engine="python")
data

,a,b,c,d
0,52,12,64,42
1,76,72,82,42
2,99,4,37,48
3,82,13,70,87
4,20,28,21,18
5,69,82,27,4
6,99,47,62,35
7,13,29,46,93
8,9,65,74,73
9,99,27,97,18


In [145]:
#find st. dev of each column
data.apply(np.std, axis=0)

a    34.388370
b    25.808719
c    23.375201
d    28.474550
dtype: float64

In [144]:
#find mean of each row
data.apply(np.mean, axis=1)

0    42.50
1    68.00
2    47.00
3    63.00
4    21.75
5    45.50
6    60.75
7    45.25
8    55.25
9    60.25
dtype: float64

In [152]:
#apply a function to every cell
data.applymap(lambda x: 1 if x<=50 else 0)

,a,b,c,d
0,0,1,0,1
1,0,0,0,1
2,0,1,1,1
3,0,1,0,0
4,1,1,1,1
5,0,0,1,1
6,0,1,0,1
7,1,1,1,0
8,1,0,0,0
9,0,1,0,1
